In [8]:
from cobra import *
from libsbml import *
import sys, getopt, os.path
from tqdm import tqdm
import zipfile
import tempfile

In [9]:
# create temporary directory
temp_dir = tempfile.TemporaryDirectory()
tmp_path = temp_dir.name
print(tmp_path)

/var/folders/qw/d0k49gb55ss0dr66vvs929d00000gn/T/tmp2an1jwt7


In [10]:
omexFile = "SARS-CoV-2_CoreModelsBALF1Epithelial.omex"
sbmlPath = omexFile.replace(".omex","")
flattenedPath = omexFile.replace(".omex","_flattened")

In [11]:
# unzip OMEX file of interest in a temporary file
with zipfile.ZipFile(omexFile, 'r') as zip_ref:
    zip_ref.extractall(tmp_path+"/"+sbmlPath)

In [12]:
# create directory to store results
os.mkdir(tmp_path+"/"+flattenedPath)

In [13]:
# convert hierarchical models defined with the SBML Level 3 Hierarchical Model Composition package 
# into a "flattened" version

for model in os.listdir(tmp_path+"/"+sbmlPath):
    
    sbmldoc = readSBML(tmp_path+"/"+sbmlPath+"/"+model)
    
    # define converter options
    props = ConversionProperties()
    props.addOption("flatten comp", True)       

    # create "flattened" version
    result = sbmldoc.convert(props)
    if result != LIBSBML_OPERATION_SUCCESS:
        sbmldoc.printErrors()
        raise SystemExit("Conversion failed... %tb")
        
    writer  = SBMLWriter()
    writer.writeSBML(sbmldoc, tmp_path+"/"+flattenedPath+"/"+model.replace(".sbml",".xml"))
    
    print(model, "Flattening....OK")


C149_C149_ACATCAGGTCGGGTCT-1_16782925330367706634.sbml Flattening....OK
C148_C148_AGCTCCTCACGAAACG-1_1152717062397344256.sbml Flattening....OK
C142_C142_CTCGTACCAAGGCTCC-1_9190006314646995726.sbml Flattening....OK


In [14]:
# correct error: '' is not a valid SBML 'SId'.
for flatObj in os.listdir(tmp_path+"/"+flattenedPath):
    
    model = io.read_sbml_model(tmp_path+"/"+flattenedPath+"/"+flatObj)
    
    # Solve problem: '' is not a valid SBML 'SId'.
    model.id = str(flatObj).replace(".xml","").replace("-","_")
    
    print(model.id, "ID correction....OK")
    
    # write corrected model (overwrite)
    io.write_sbml_model(model,tmp_path+"/"+flattenedPath+"/"+flatObj)

'' is not a valid SBML 'SId'.


C148_C148_AGCTCCTCACGAAACG_1_1152717062397344256 ID correction....OK


'' is not a valid SBML 'SId'.


C149_C149_ACATCAGGTCGGGTCT_1_16782925330367706634 ID correction....OK


'' is not a valid SBML 'SId'.


C142_C142_CTCGTACCAAGGCTCC_1_9190006314646995726 ID correction....OK


Final models are stored in: **tmp_path/flattenedPath/**

### The final models can be used with any COBRApy or COBRAToolbox function.

In [ ]:
# delete temporary directory if no longer needed
temp_dir.cleanup()